In [ ]:
from __future__ import annotations

import os

os.environ["RERUN_NOTEBOOK_ASSET"] = "serve-local"

import os

import cv2
import pyarrow as pa
import requests
import rerun as rr

In [ ]:
def download(url: str):
    if not os.path.exists("datasets"):
        os.makedirs("datasets")
    filename = url.split("/")[-1]
    dest = f"datasets/{filename}"
    if not os.path.exists(dest):
        with open(dest, "wb") as f:
            f.write(requests.get(url).content)

In [ ]:
# OPF
download("http://app.rerun.io/version/0.18.2/examples/open_photogrammetry_format.rrd")
opf = rr.dataframe.load_recording("datasets/open_photogrammetry_format.rrd")

In [ ]:
# What timelines do we have?
opf.schema().index_columns()

In [ ]:
# What components do we have?
opf.schema().component_columns()

In [ ]:
# Extract a single point-cloud

cloud = rr.dataframe.ComponentColumnSelector("world/points", rr.components.Position3D)

batches = opf.view(index="image", contents="world/points").select([cloud])

# Should the python APIs do this for us?
table = pa.Table.from_batches(batches)

print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

In [ ]:
cloud = table[0][0]
type(cloud)

In [ ]:
points_np = cloud.values.flatten().to_numpy().reshape(-1, 3)

In [ ]:
points_np[:10]

In [ ]:
# Find the positions of all the camera estimates

img_num = rr.dataframe.TimeColumnSelector("image")
logged = rr.dataframe.TimeColumnSelector("log_time")

pos = rr.dataframe.ComponentColumnSelector("world/cameras", rr.components.Translation3D)
quat = rr.dataframe.ComponentColumnSelector("world/cameras", rr.components.TransformMat3x3)

view = opf.view(index="image", contents="world/cameras")
batches = view.select([img_num, logged, pos, quat])

table = pa.Table.from_batches(batches)

print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

In [ ]:
# Annoying -- row traversal even though they were logged together :-(
table = table.combine_chunks()
print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

In [ ]:
# Look at some samples (Polars would make this nicer)
table.take([0, 10])

In [ ]:
# TODO: Let's feed this into a geometry library

In [ ]:
# Maybe we only wanted to query poses for a few images
# Find the positions of all the camera estimates

batches = view.filter_range_sequence(start=10, end=20).select([img_num, logged, pos, quat])

table = pa.Table.from_batches(batches)

print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

In [ ]:
table.take([0, 10])

In [ ]:
# TODO: latest-at

In [ ]:
# Let's grab an image

view = opf.view(index="image", contents="/**").filter_range_sequence(42, 42)

img_num = rr.dataframe.TimeColumnSelector("image")

# Getting these right is annoying -- would be nice to support '/**' somehow
blob = rr.dataframe.ComponentColumnSelector("world/cameras/image", rr.components.Blob)
media = rr.dataframe.ComponentColumnSelector("world/cameras/image", rr.components.MediaType)

batches = view.select([blob, media])

table = pa.Table.from_batches(batches)

print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

In [ ]:
# ^^ when we typo something, we just find no data
table.schema

In [ ]:
# Let's grab an image for real this time

# Getting these right is annoying -- would be nice to support '/**' somehow
blob = rr.dataframe.ComponentColumnSelector("world/cameras/image/rgb", rr.components.Blob)
media = rr.dataframe.ComponentColumnSelector("world/cameras/image/rgb", rr.components.MediaType)

batches = view.select([blob, media])

table = pa.Table.from_batches(batches)

print(f"Found {table.num_rows} rows in {len(table.columns[0].chunks)} chunks")

## NOTE: Kind of surprising to find 2 rows here. The first one gets included because the VIEW contains
## Contains static data.

In [ ]:
print(f"Image is encoded as: {table[1][1][0]}")

In [ ]:
image = image = cv2.imdecode(table[0][1][0].values.to_numpy(), cv2.IMREAD_COLOR)

In [ ]:
rr.init("preview")

rr.log("img", rr.Image(image, color_model="bgr"))

rr.notebook_show()

In [ ]:
# BGR shader not working on web in firefox :sob:

In [ ]:
rr.init("preview")

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

rr.log("img", rr.Image(image))

rr.notebook_show()